# study2 모델. 차량 파손 여부 탐지 + ver2.0(kaggle data 추가) + ver3.0(ai-hub 12000장)

 ## 1. 데이터셋 구축 워크플로우 (Workflow)
 #### 1) 준비 (Preparation)
  - 입력 데이터:
    - Damaged 폴더 (파손 차량 이미지 + 파손 위치 라벨 JSON)
    - Normal 폴더 (정상 차량 이미지)
 - 도구 (Tool):
    - Study 1 모델: 이미 학습된 차량 탐지 모델 (yolov8_fine_tuning.pt)
 #### 2) 데이터 분할 (Splitting)
  - 비율: 전체 데이터를 Train (70%) : Val (20%) : Test (10%) 비율로 무작위 분할
  - 목적: 학습용, 검증용, 평가용 데이터를 미리 나누어 데이터 유출(Data Leakage)을 방지
 #### 3) 핵심 처리 과정 (Core Process) - 반복문(Loop)
  - 차량 탐지 (Detect Vehicle):
    - 1단계 모델을 사용하여 이미지에서 **차량의위치BBox**를 찾음
    - (만약 차량을 못 찾으면? → 원본 이미지 전체를 사용하도록 Fallback)
 - 영역 자르기 (Crop with Margin):
    - 차량 BBox에 **여유공간(Margin15%)** 을 더해서 이미지를 잘라냄.
    - 이때, 이미지 밖으로 나가는 좌표는 **Clamp(울타리)** 처리하여 이미지 안쪽으로 맞춤
 - 크기 조정 (Resize):
    - 잘라낸 이미지를 모델 입력 크기(예: 640x640 또는 1024x1024)로 변경
 - 라벨 변환 (Label Remapping):
    - Damaged: 원본 이미지 기준의 파손 좌표 (x, y)를 Crop된 이미지 기준의 상대 좌표로 변환.(이때도 좌표가 튀지 않게 Clamp 적용 필수)
    - Normal: 파손이 없으므로 **빈 텍스트 파일.txt**을 생성 (Negative Sample 학습용)
 #### 4) 저장 (Saving)
 - 변환된 이미지와 라벨(.txt)을 YOLO 학습 포맷에 맞춰 저장
    - images/train,
    - labels/trainimages/val,
    - labels/valimages/test, labels/test
 - **클래스 통합**: 데이터 복잡도를 줄이기 위해 모든 파손 종류(Scratch, Dent 등)를 'Damage' (ID: 0) 하나로 통합
 #### 5) 검증 (Verification)
 - 시각화: 변환된 데이터(Test Set) 중 3장을 무작위로 뽑아, 파손 박스가 차량 위에 정확히 그려졌는지 눈으로 확인

In [ ]:
### train시 1시간 정도 소모 됩니다. colab환경에서 아래를 browser의 console 에서 붙여 넣기가 필요할 수 있습니다.
### shift+cntr+i 로 browser console 열기
# https://github.com/chulminkw/DLCV/blob/master/data/util/colab_autoclick.js
'''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''

# 라이브러리 설치

In [ ]:
# [Cell 1] 라이브러리 설치
# YOLOv8 및 데이터 처리에 필요한 라이브러리 설치
!pip install ultralytics tqdm

import os
import shutil
import json
import random
import yaml
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from ultralytics import YOLO
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import drive
import glob
import cv2
from sklearn.model_selection import train_test_split

# 구글 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

print("✅ 환경 설정 완료")

In [ ]:
import os

# 1. 압축을 풀 폴더 생성 (파일이 섞이지 않게 폴더를 따로 만드는 것이 좋습니다)
extract_path = "/content/dataset"
os.makedirs(extract_path, exist_ok=True)

# 2. 압축 해제 실행
# -q: Quiet 모드 (로그 숨김 -> 속도 향상)
# -d: Destination (저장할 경로 지정)
!unzip -q /content/AI_HUB_DAMAGE_DATASET.zip -d {extract_path}
!unzip -q /content/RAW.zip -d {extract_path}

print(f"✅ 압축 해제 완료! 경로: {extract_path}")

# 데이터셋 구축 및 분할

In [ ]:
import numpy as np
# =========================================================
# ⚙️ [설정] 경로 및 파라미터 (수정됨)
# =========================================================
# 1. 원본 데이터 경로 (변경된 부분)
# 1. 입력 데이터 경로 (Source)
# (1) AI-Hub (Damaged)
#AI_HUB_IMG_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/AI_HUB_DAMAGE_DATASET/images" # 속도 개선을 위해 content에 직접 업로드
#AI_HUB_LBL_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/AI_HUB_DAMAGE_DATASET/labels"
AI_HUB_IMG_ROOT = "/content/dataset/AI_HUB_DAMAGE_DATASET/images" # 속도 개선을 위해 content에 직접 업로드
AI_HUB_LBL_ROOT = "/content/dataset/AI_HUB_DAMAGE_DATASET/labels"

# (2) Normal & Background (Source Base)
#NORMAL_ROOT_BASE = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/RAW"
NORMAL_ROOT_BASE = "/content/dataset/RAW"


# 2. 저장할 데이터셋 경로
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"

# 3. 1단계 차량 탐지 모델 경로 (★ 중요: 실제 pt 파일 경로가 맞는지 확인 필요)
MODEL_PATH = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/FINE_TUNING_MODEL/yolov8x_fine_tuning_5th/weights/best.pt"

# 4. 파라미터
IMG_SIZE = 640       # Stage 2 입력 사이즈
MARGIN_RATIO = 0.15  # Crop 마진
CONF_THRES = 0.1     # [요청반영] 차량 탐지 임계값 (Recall 100% 기준)
SEED = 42

random.seed(SEED)
np.random.seed(SEED)

print("✅ 설정 및 경로 정의 완료")

In [ ]:
# [Cell 2] 유틸리티 함수 정의 (Clamp, 좌표 변환 - 기존 유지)


def clamp(value, min_val, max_val):
    return max(min_val, min(value, max_val))

def get_vehicle_crop_box(image, model, margin=0.15):
    """Stage 1 모델로 차량을 찾아 Crop 좌표 반환 (conf=0.1 적용)"""
    h_img, w_img = image.shape[:2]

    # 1. 추론
    results = model(image, verbose=False, conf=CONF_THRES)

    best_box = None
    max_area = 0

    for r in results:
        for box in r.boxes:
            coords = box.xyxy[0].cpu().numpy() # x1, y1, x2, y2
            area = (coords[2] - coords[0]) * (coords[3] - coords[1])
            if area > max_area:
                max_area = area
                best_box = coords

    # 2. Fallback (미탐지 시 전체 이미지 사용 - 근접샷 대응)
    if best_box is None:
        return 0, 0, w_img, h_img, False

    # 3. Margin 적용 & Clamp
    x1, y1, x2, y2 = best_box
    w_box = x2 - x1; h_box = y2 - y1

    nx1 = int(clamp(x1 - w_box * margin, 0, w_img))
    ny1 = int(clamp(y1 - h_box * margin, 0, h_img))
    nx2 = int(clamp(x2 + w_box * margin, 0, w_img))
    ny2 = int(clamp(y2 + h_box * margin, 0, h_img))

    return nx1, ny1, nx2, ny2, True

def convert_label_global_to_local(src_txt_path, crop_coords, src_size):
    """
    [핵심] 원본 TXT 라벨을 읽어 Crop된 이미지 기준 좌표로 변환
    src_txt_path: 원본 라벨 경로
    crop_coords: (cx1, cy1, cx2, cy2) - 크롭 영역
    src_size: (w_org, h_org) - 원본 이미지 크기
    """
    if not os.path.exists(src_txt_path):
        return []

    cx1, cy1, cx2, cy2 = crop_coords
    w_org, h_org = src_size
    crop_w = cx2 - cx1
    crop_h = cy2 - cy1

    new_labels = []

    with open(src_txt_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5: continue

        cls = int(parts[0]) # 클래스 ID (여기서는 모두 0으로 통합 예정이면 나중에 처리)
        # 원본 정규화 좌표
        n_cx, n_cy, n_w, n_h = map(float, parts[1:5])

        # 1. Denormalize (픽셀 좌표로 변환)
        gx_center = n_cx * w_org
        gy_center = n_cy * h_org
        gw = n_w * w_org
        gh = n_h * h_org

        gx1 = gx_center - gw / 2
        gy1 = gy_center - gh / 2
        gx2 = gx_center + gw / 2
        gy2 = gy_center + gh / 2

        # 2. Intersection (Crop 영역과 교차하는 부분 계산)
        # 라벨 박스가 크롭 영역 밖으로 나갔는지 확인
        ix1 = max(gx1, cx1)
        iy1 = max(gy1, cy1)
        ix2 = min(gx2, cx2)
        iy2 = min(gy2, cy2)

        # 교차 영역이 유효하지 않으면(잘려 나감) 스킵
        if ix2 <= ix1 or iy2 <= iy1:
            continue

        # 3. Remapping (Crop 영역 기준 좌표로 변환)
        lx1 = ix1 - cx1
        ly1 = iy1 - cy1
        lx2 = ix2 - cx1
        ly2 = iy2 - cy1

        lw = lx2 - lx1
        lh = ly2 - ly1

        # 너무 작은 박스는 제거 (노이즈 방지)
        if lw < 2 or lh < 2: continue

        # 4. Normalize (Crop 이미지 크기 기준)
        final_cx = (lx1 + lw / 2) / crop_w
        final_cy = (ly1 + lh / 2) / crop_h
        final_w = lw / crop_w
        final_h = lh / crop_h

        # 값 클램핑 (0~1 사이 안전장치)
        final_cx = clamp(final_cx, 0, 1)
        final_cy = clamp(final_cy, 0, 1)
        final_w = clamp(final_w, 0, 1)
        final_h = clamp(final_h, 0, 1)

        # 클래스 ID는 0 (Damage)으로 통합
        new_labels.append(f"0 {final_cx:.6f} {final_cy:.6f} {final_w:.6f} {final_h:.6f}")

    return new_labels

def process_and_save(src_img_path, src_lbl_path, dst_img_path, dst_txt_path, model):
    img = cv2.imread(src_img_path)
    if img is None: return False

    h_org, w_org = img.shape[:2]

    # 1. Crop 좌표 계산
    cx1, cy1, cx2, cy2, detected = get_vehicle_crop_box(img, model, MARGIN_RATIO)
    crop_w = cx2 - cx1
    crop_h = cy2 - cy1

    if crop_w <= 0 or crop_h <= 0: return False

    # 2. 이미지 Crop & Resize
    crop_img = img[cy1:cy2, cx1:cx2]
    resized_img = cv2.resize(crop_img, (IMG_SIZE, IMG_SIZE))

    # 3. 라벨 변환 (Damaged인 경우만)
    final_labels = []
    if src_lbl_path: # Damaged (Normal은 None)
        final_labels = convert_label_global_to_local(
            src_lbl_path, (cx1, cy1, cx2, cy2), (w_org, h_org)
        )

    # 4. 저장
    cv2.imwrite(dst_img_path, resized_img)
    with open(dst_txt_path, 'w') as f:
        if final_labels:
            f.write("\n".join(final_labels))

    return True

print("✅ 함수 및 설정 완료")

In [ ]:
# [Cell 2] 데이터셋 구축 메인 프로세스

def build_dataset_stage2():
    # 0. 초기화
    if os.path.exists(DATASET_ROOT):
        shutil.rmtree(DATASET_ROOT)

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(DATASET_ROOT, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(DATASET_ROOT, 'labels', split), exist_ok=True)

    # 1. 모델 로드
    if not os.path.exists(MODEL_PATH):
        print(f"❌ 모델 없음: {MODEL_PATH}")
        return
    model = YOLO(MODEL_PATH)

    # 2. 파일 리스트 수집
    print("📂 파일 수집 중...")
    extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']

    # (1) Damaged Data
    d_files = []
    for ext in extensions:
        d_files.extend(glob.glob(os.path.join(AI_HUB_IMG_ROOT, "**", ext), recursive=True))
    d_files = list(set(d_files))

    # (2) Normal Data (Background 제외)
    n_files = []
    target_folders = ['normal', 'normal(kaggle_dataset)']
    for folder in target_folders:
        path = os.path.join(NORMAL_ROOT_BASE, folder)
        if os.path.exists(path):
            for ext in extensions:
                n_files.extend(glob.glob(os.path.join(path, "**", ext), recursive=True))
    n_files = list(set(n_files))

    print(f"   - Damaged: {len(d_files)}장")
    print(f"   - Normal : {len(n_files)}장")

    # 3. 데이터 분할
    def split_list(files):
        if not files: return [], [], []
        train, test = train_test_split(files, test_size=0.1, random_state=SEED)
        train, val = train_test_split(train, test_size=0.2222, random_state=SEED)
        return train, val, test

    dt, dv, dts = split_list(d_files)
    nt, nv, nts = split_list(n_files)

    # 4. 처리 루프
    tasks = [
        ('train', dt, 'damaged'), ('val', dv, 'damaged'), ('test', dts, 'damaged'),
        ('train', nt, 'normal'),  ('val', nv, 'normal'),  ('test', nts, 'normal')
    ]

    total_stats = {'train': 0, 'val': 0, 'test': 0}

    print("⚡ Crop 및 데이터셋 생성 시작...")

    for split, files, kind in tasks:
        for file_path in tqdm(files, desc=f"[{split}] {kind}"):
            filename = os.path.basename(file_path)
            basename = os.path.splitext(filename)[0]

            dst_img = os.path.join(DATASET_ROOT, 'images', split, filename)
            dst_txt = os.path.join(DATASET_ROOT, 'labels', split, basename + ".txt")

            src_lbl = None
            if kind == 'damaged':
                # TXT 라벨 찾기 (재귀 검색은 느리므로 경로 매핑 시도)
                # 1. AI_HUB_LBL_ROOT + 상대경로 (가장 빠름)
                try:
                    rel_path = os.path.relpath(os.path.dirname(file_path), AI_HUB_IMG_ROOT)
                    candidate = os.path.join(AI_HUB_LBL_ROOT, rel_path, basename + ".txt")
                    if os.path.exists(candidate):
                        src_lbl = candidate
                except: pass

                # 2. 없으면 LBL_ROOT 바로 아래 확인
                if src_lbl is None:
                    candidate = os.path.join(AI_HUB_LBL_ROOT, basename + ".txt")
                    if os.path.exists(candidate):
                        src_lbl = candidate

                # 3. 그래도 없으면 스킵 (라벨 없는 손상 데이터는 의미 없음)
                if src_lbl is None:
                    continue

            # 처리 실행
            if process_and_save(file_path, src_lbl, dst_img, dst_txt, model):
                total_stats[split] += 1

    # 5. YAML 생성
    yaml_txt = f"""
path: {DATASET_ROOT}
train: images/train
val: images/val
test: images/test
nc: 1
names: ['Damage']
"""
    with open(os.path.join(DATASET_ROOT, "data_damage_v2.yaml"), "w") as f:
        f.write(yaml_txt)

    print("\n✅ 완료!")
    print(f"   Total Generated: {sum(total_stats.values())} files")
    print(f"   YAML Path: {os.path.join(DATASET_ROOT, 'data_damage_v2.yaml')}")

# 실행
build_dataset_stage2()

## 시각화 (검증)

In [ ]:
DATASET_ROOT

In [ ]:
# [Cell 4] 시각화 (검증)
def visualize_random_samples(num_samples=3):
    """
    생성된 데이터셋(Test Set) 중 Damaged 라벨이 있는 이미지를 랜덤으로 골라 시각화
    """
    test_img_dir = os.path.join(DATASET_ROOT, 'images', 'test')
    test_lbl_dir = os.path.join(DATASET_ROOT, 'labels', 'test')

    # 1. 라벨 파일이 있는(내용이 있는) 파일 찾기
    candidates = []
    if not os.path.exists(test_lbl_dir):
        print("테스트 라벨 디렉토리가 없습니다.")
        return

    for txt_file in os.listdir(test_lbl_dir):
        txt_path = os.path.join(test_lbl_dir, txt_file)
        if os.path.getsize(txt_path) > 0: # 내용이 있으면 Damage가 있는 것
            candidates.append(txt_file)

    if not candidates:
        print("시각화할 파손 데이터가 없습니다.")
        return

    # 2. 랜덤 샘플링
    samples = random.sample(candidates, min(len(candidates), num_samples))

    plt.figure(figsize=(15, 5))

    for i, txt_name in enumerate(samples):
        basename = os.path.splitext(txt_name)[0]
        img_name = basename + ".jpg" # 확장자는 jpg라고 가정
        if not os.path.exists(os.path.join(test_img_dir, img_name)):
            img_name = basename + ".png"

        img_path = os.path.join(test_img_dir, img_name)
        txt_path = os.path.join(test_lbl_dir, txt_name)

        # 이미지 로드
        img = cv2.imread(img_path)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # 박스 그리기
        with open(txt_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls, cx, cy, bw, bh = map(float, parts[:5])

                    # YOLO -> Pixel
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)

                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img, "Damage", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        plt.subplot(1, num_samples, i+1)
        plt.imshow(img)
        plt.title(f"Sample {i+1}\n{basename}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# 실행
print("🔍 생성된 데이터셋 검증 (Random 3 Samples)")
visualize_random_samples()

In [ ]:
import os
import glob
from pathlib import Path
from tqdm import tqdm

# =========================================================
# [설정] 분석할 데이터셋 루트 경로
# =========================================================
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"

def analyze_dataset_status(root_dir):
    print(f"📊 데이터셋 현황 분석 보고서")
    print(f"📂 타겟 경로: {root_dir}")
    print("=" * 60)

    if not os.path.exists(root_dir):
        print("❌ 오류: 해당 경로가 존재하지 않습니다.")
        return

    splits = ['train', 'val', 'test']
    total_summary = {'images': 0, 'labels': 0, 'damaged': 0, 'normal': 0}

    # 헤더 출력
    print(f"{'Split':<10} | {'Images':<10} | {'Labels':<10} | {'Damaged(Box)':<12} | {'Normal(Empty)':<12} | {'Status'}")
    print("-" * 80)

    for split in splits:
        img_dir = os.path.join(root_dir, 'images', split)
        lbl_dir = os.path.join(root_dir, 'labels', split)

        # 1. 디렉토리 존재 확인
        if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
            print(f"{split:<10} | {'MISSING DIR':<30} | ❌ Check Path")
            continue

        # 2. 파일 리스트업
        # 이미지 확장자 대소문자 모두 포함
        img_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
        img_files = []
        for ext in img_extensions:
            img_files.extend(glob.glob(os.path.join(img_dir, ext)))

        lbl_files = glob.glob(os.path.join(lbl_dir, "*.txt"))

        # 3. 상세 분석 (Damaged vs Normal)
        # Damaged: txt 파일 안에 내용(좌표)이 있음
        # Normal: txt 파일이 없거나, 비어 있음

        n_damaged = 0 # 박스가 있는 파일
        n_normal = 0  # 빈 파일 (또는 라벨 파일 없음)

        # 파일명(stem) 기준 매핑을 위해 set 생성
        img_stems = {Path(f).stem for f in img_files}
        lbl_stems = {Path(f).stem for f in lbl_files}

        # 라벨 파일 내용을 확인하여 분류
        for lbl_path in lbl_files:
            if os.path.getsize(lbl_path) > 0:
                n_damaged += 1
            else:
                n_normal += 1

        # 라벨 파일 자체가 없는 경우도 Normal로 간주할 것인지 체크
        # (현재 구축 로직상 Normal도 빈 txt를 생성하므로, 라벨 파일 수 == 이미지 수가 이상적)
        missing_labels = len(img_stems - lbl_stems)
        if missing_labels > 0:
             # 라벨 파일이 아예 없는 경우 (보통은 Normal로 취급하거나 오류)
             # 여기서는 단순히 개수로만 파악
             pass

        # 통계 집계
        n_imgs = len(img_files)
        n_lbls = len(lbl_files)

        total_summary['images'] += n_imgs
        total_summary['labels'] += n_lbls
        total_summary['damaged'] += n_damaged
        total_summary['normal'] += n_normal + missing_labels # 라벨 없는 것도 Normal 범주로 일단 포함

        # 상태 판정
        if n_imgs == n_lbls:
            status = "✅ Perfect"
        elif n_imgs > n_lbls:
            status = f"⚠️ Miss {n_imgs - n_lbls} lbls"
        else:
            status = f"❓ Extra {n_lbls - n_imgs} lbls"

        # 출력
        print(f"{split:<10} | {n_imgs:<10} | {n_lbls:<10} | {n_damaged:<12} | {n_normal:<12} | {status}")

    print("-" * 80)
    print(f"📌 [종합 요약]")
    print(f"   - 총 이미지 수 : {total_summary['images']} 장")
    print(f"   - 총 라벨 파일 : {total_summary['labels']} 개")
    print(f"   --------------------------------")
    print(f"   🚗 손상 차량 (Positive) : {total_summary['damaged']} 장")
    print(f"   🟩 정상/배경 (Negative) : {total_summary['normal']} 장")

    # YAML 존재 확인
    yaml_path = os.path.join(root_dir, "data_damage_v2.yaml")
    if not os.path.exists(yaml_path):
        # 이름이 다를 수도 있으니 검색
        yamls = glob.glob(os.path.join(root_dir, "*.yaml"))
        if yamls:
            print(f"   📄 YAML 파일 : ✅ 발견됨 ({os.path.basename(yamls[0])})")
        else:
            print(f"   📄 YAML 파일 : ❌ 없음 (확인 필요)")
    else:
        print(f"   📄 YAML 파일 : ✅ 발견됨 (data_damage_v2.yaml)")

# 실행
analyze_dataset_status(DATASET_ROOT)

# 모델 학습 1차

In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# =========================================================
# [설정] 경로
# =========================================================
# 원본 (구글 드라이브)
SOURCE_DIR = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"
# 목적지 (Colab 로컬)
DEST_DIR = "/content/temp"

def copy_file(src_path, dst_path):
    shutil.copy2(src_path, dst_path)

def fast_copy_with_progress(src, dst):
    if not os.path.exists(src):
        print(f"❌ 원본 경로가 없습니다: {src}")
        return

    print("🔍 파일 목록을 스캔 중입니다... (잠시만 기다려주세요)")

    # 1. 모든 파일 리스트업
    files_to_copy = []
    for root, dirs, files in os.walk(src):
        # 목적지 폴더 구조 미리 생성
        rel_path = os.path.relpath(root, src)
        dest_path = os.path.join(dst, rel_path)
        os.makedirs(dest_path, exist_ok=True)

        for f in files:
            src_file = os.path.join(root, f)
            dst_file = os.path.join(dest_path, f)
            files_to_copy.append((src_file, dst_file))

    total_files = len(files_to_copy)
    print(f"📦 총 복사할 파일 수: {total_files}개")
    print(f"🚀 멀티쓰레딩 복사 시작 (8 workers)...")

    # 2. 병렬 복사 실행 (Thread Pool)
    with ThreadPoolExecutor(max_workers=8) as executor:
        # tqdm으로 진행바 연결
        list(tqdm(executor.map(lambda x: copy_file(x[0], x[1]), files_to_copy),
                  total=total_files,
                  unit='file'))

    print("\n✅ 데이터셋 복사 완료!")

# 실행
if os.path.exists(DEST_DIR):
    shutil.rmtree(DEST_DIR) # 기존꺼 삭제 후 재시작

fast_copy_with_progress(SOURCE_DIR, DEST_DIR)

In [ ]:

import yaml
# =========================================================
# 2. [설정] YAML 파일 경로 재지정
# =========================================================
# 로컬로 복사했으므로 YAML 파일 내의 path도 수정해줘야 함
yaml_path_local = os.path.join(DEST_DIR, 'data_damage_v2.yaml')

# YAML 내용 수정 (path: /content/temp_dataset 으로 변경)
with open(yaml_path_local, 'r') as f:
    data = yaml.safe_load(f)

data['path'] = DEST_DIR  # 경로를 로컬로 변경

with open(yaml_path_local, 'w') as f:
    yaml.dump(data, f)

print(f"📂 학습용 YAML 경로: {yaml_path_local}")

In [ ]:
import torch
import gc

# 메모리 청소
gc.collect()
torch.cuda.empty_cache()

In [ ]:

from ultralytics import YOLO

# =========================================================
# 3. [설정] 저장 경로 및 모델 학습
# =========================================================
PROJECT_DIR = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"
SAVE_DIR = os.path.join(PROJECT_DIR, "FINE_TUNING_MODEL")

def train_model():
    model = YOLO('yolov8m.pt')

    print("🔥 Stage 2 학습 시작...")
    model.train(
        data=yaml_path_local, # 로컬 YAML 사용
        epochs=100,
        imgsz=640,
        batch=32,

        # [튜닝 포인트 반영]
        patience=25,       # 10은 너무 짧습니다. 25로 늘리세요.
        optimizer='AdamW',
        lr0=1e-4,
        cos_lr=True,       # 학습률 스케줄러 추가 (권장)
        single_cls=True,   # 모든 라벨을 Damage 하나로 통합 (권장)

        close_mosaic=10,   # 오타 수정됨 (OK)

        project=SAVE_DIR,
        name='yolov8m_damage_stage2_3rd',
        exist_ok=True,
        verbose=True
    )
    print("🎉 학습 완료!")
    return os.path.join(SAVE_DIR, 'yolov8m_damage_stage2_3rd', 'weights', 'best.pt')

# 학습 실행
best_model_path = train_model()
print(f"🏆 Best Model Path: {best_model_path}")

## 성능 테스트(conf 0.25)

In [ ]:
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm

# [Cell 3] 성능 평가 (Detection & Classification)

best_model_path = '/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/FINE_TUNING_MODEL/yolov8m_damage_stage2_3rd/weights/best.pt'
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"
# YAML 파일 경로
YAML_PATH = os.path.join(DATASET_ROOT, 'data_damage_v2.yaml')

def evaluate_performance(model_path):
    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_path}")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {model_path}")

    model = YOLO(model_path)

    # ---------------------------------------------------------
    # 1. 객체 탐지 성능 평가 (Detection Metrics: mAP)
    # ---------------------------------------------------------
    print("\n📊 [1] BBox 탐지 성능 (mAP) 계산 중...")
    # split='test'로 설정하여 Test Set에 대해 평가
    metrics = model.val(data=YAML_PATH, split='test', verbose=False)

    map50 = metrics.box.map50
    map5095 = metrics.box.map

    # ---------------------------------------------------------
    # 2. 이미지 분류 성능 & 속도 평가 (Classification & Speed)
    # ---------------------------------------------------------
    print("📊 [2] 이미지 분류 성능 및 추론 속도 측정 중...")

    TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
    TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    results_list = []
    speed_stats = []

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        txt_file = os.path.splitext(file)[0] + ".txt"
        lbl_path = os.path.join(TEST_LBL_DIR, txt_file)

        # A. 정답 확인 (라벨 파일 내용 유무)
        true_label = 0
        if os.path.exists(lbl_path) and os.path.getsize(lbl_path) > 0:
            true_label = 1 # Damaged

        # B. 모델 추론
        results = model(img_path, conf=0.25, verbose=False)

        # C. 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # D. 예측 라벨 (박스 유무)
        pred_label = 0
        if len(results[0].boxes) > 0:
            pred_label = 1 # Damaged

        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label)
        })

    df = pd.DataFrame(results_list)

    # =========================================================
    # [최종 분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # 1. mAP 결과 출력
    print(f"🎯 1. 객체 탐지 성능 (Detection Metrics):")
    print(f"   - mAP@50    : {map50:.4f}")
    print(f"   - mAP@50-95 : {map5095:.4f}")

    # 2. 정확도 출력
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"\n✅ 2. 분류 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # 3. 속도 출력

    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time if avg_time > 0 else 0

        print(f"\n⚡ 3. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # 4. 상세 리포트
    print(f"\n📝 4. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Normal', 'Damaged']))

    # 5. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Normal', 'Pred: Damaged'],
                yticklabels=['True: Normal', 'True: Damaged'])
    plt.title(f'Confusion Matrix\n(Damage Detection)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # 결과 저장
    save_path = os.path.join(RESULT_ROOT, "inference_finetuned_3rd_test_results_conf025.csv")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 실행 결과 저장 완료: {save_path}")

# 평가 실행
evaluate_performance(best_model_path)


## 오탐 대상 시각화

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

# =========================================================
# [설정] 경로 (이전 단계와 동일하게 설정)
# =========================================================

DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"

# 모델 및 결과 파일 경로
SAVE_DIR = os.path.join(PROJECT_ROOT, "FINE_TUNING_MODEL")
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8m_damage_stage2_3rd', 'weights', 'best.pt')

RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_finetuned_3rd_test_results_conf025.csv")

# 테스트 이미지 및 라벨 경로
TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

def visualize_failures_batch(start_index=0, batch_size=3):
    """
    오탐 데이터를 start_index부터 batch_size만큼 시각화하는 함수
    """
    # 1. 결과 CSV 로드 및 오탐 필터링
    if not os.path.exists(RESULT_CSV_PATH):
        print(f"❌ 결과 파일이 없습니다: {RESULT_CSV_PATH}")
        return

    df = pd.read_csv(RESULT_CSV_PATH)
    failures = df[df['is_correct'] == False] # 오탐만 추출

    total_failures = len(failures)
    if total_failures == 0:
        print("🎉 오탐 데이터가 없습니다! 완벽합니다.")
        return

    # 인덱스 범위 체크
    if start_index >= total_failures:
        print(f"⚠️ 더 이상 확인할 오탐 데이터가 없습니다. (총 {total_failures}건)")
        return

    end_index = min(start_index + batch_size, total_failures)
    target_samples = failures.iloc[start_index : end_index]

    print(f"🔍 오탐 데이터 시각화: {start_index+1} ~ {end_index} / 총 {total_failures}건")

    # 2. 모델 로드 (박스 시각화용)
    model = YOLO(MODEL_PATH)

    # 3. 시각화
    plt.figure(figsize=(15, 6 * len(target_samples)))

    for i, (_, row) in enumerate(target_samples.iterrows()):
        filename = row['filename']
        true_label = row['true_label'] # 0: Normal, 1: Damaged
        img_path = os.path.join(TEST_IMG_DIR, filename)
        lbl_path = os.path.join(TEST_LBL_DIR, os.path.splitext(filename)[0] + ".txt")

        # 이미지 로드
        img = cv2.imread(img_path)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # A. 정답(Ground Truth) 박스 그리기 (초록색)
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls, cx, cy, bw, bh = map(float, line.strip().split())
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(img, "GT", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # B. 모델 예측(Prediction) 박스 그리기 (빨간색)
        # 시각화를 위해 다시 추론 수행
        results = model(img_path, conf=0.25, verbose=False)
        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
                cv2.putText(img, f"Pred {conf:.2f}", (x1, y2+25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # 타이틀 설정
        str_true = "Damaged" if true_label == 1 else "Normal"
        str_pred = "Normal" if true_label == 1 else "Damaged" # 오탐이므로 반대
        fail_type = "False Negative (미탐)" if true_label == 1 else "False Positive (오탐/과탐)"

        plt.subplot(len(target_samples), 1, i+1)
        plt.imshow(img)
        plt.title(f"[{start_index+i+1}] {filename}\nType: {fail_type} | GT: {str_true} (Green) vs Pred: {str_pred} (Red)",
                  fontsize=14, color='red', fontweight='bold')
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# 모델 및 결과 파일 경로
SAVE_DIR = os.path.join(PROJECT_ROOT, "FINE_TUNING_MODEL")
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8m_damage_stage2_3rd', 'weights', 'best.pt')

RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_finetuned_3rd_test_results_conf025.csv")

visualize_failures_batch(start_index=0, batch_size=100)

## 성능 테스트(conf 0.25->0.1)

In [ ]:
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm

# [Cell 3] 성능 평가 (Detection & Classification)

best_model_path = '/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/FINE_TUNING_MODEL/yolov8m_damage_stage2_3rd/weights/best.pt'
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"
# YAML 파일 경로
YAML_PATH = os.path.join(DATASET_ROOT, 'data_damage_v2.yaml')

def evaluate_performance(model_path):
    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_path}")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {model_path}")

    model = YOLO(model_path)

    # ---------------------------------------------------------
    # 1. 객체 탐지 성능 평가 (Detection Metrics: mAP)
    # ---------------------------------------------------------
    print("\n📊 [1] BBox 탐지 성능 (mAP) 계산 중...")
    # split='test'로 설정하여 Test Set에 대해 평가
    metrics = model.val(data=YAML_PATH, split='test', verbose=False)

    map50 = metrics.box.map50
    map5095 = metrics.box.map

    # ---------------------------------------------------------
    # 2. 이미지 분류 성능 & 속도 평가 (Classification & Speed)
    # ---------------------------------------------------------
    print("📊 [2] 이미지 분류 성능 및 추론 속도 측정 중...")

    TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
    TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    results_list = []
    speed_stats = []

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        txt_file = os.path.splitext(file)[0] + ".txt"
        lbl_path = os.path.join(TEST_LBL_DIR, txt_file)

        # A. 정답 확인 (라벨 파일 내용 유무)
        true_label = 0
        if os.path.exists(lbl_path) and os.path.getsize(lbl_path) > 0:
            true_label = 1 # Damaged

        # B. 모델 추론
        results = model(img_path, conf=0.1, verbose=False)

        # C. 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # D. 예측 라벨 (박스 유무)
        pred_label = 0
        if len(results[0].boxes) > 0:
            pred_label = 1 # Damaged

        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label)
        })

    df = pd.DataFrame(results_list)

    # =========================================================
    # [최종 분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # 1. mAP 결과 출력
    print(f"🎯 1. 객체 탐지 성능 (Detection Metrics):")
    print(f"   - mAP@50    : {map50:.4f}")
    print(f"   - mAP@50-95 : {map5095:.4f}")

    # 2. 정확도 출력
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"\n✅ 2. 분류 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # 3. 속도 출력

    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time if avg_time > 0 else 0

        print(f"\n⚡ 3. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # 4. 상세 리포트
    print(f"\n📝 4. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Normal', 'Damaged']))

    # 5. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Normal', 'Pred: Damaged'],
                yticklabels=['True: Normal', 'True: Damaged'])
    plt.title(f'Confusion Matrix\n(Damage Detection)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # 결과 저장
    save_path = os.path.join(RESULT_ROOT, "inference_finetuned_3rd_test_results_conf010.csv")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 실행 결과 저장 완료: {save_path}")

# 평가 실행
evaluate_performance(best_model_path)


## 오탐 대상 시각화

In [ ]:
# 모델 및 결과 파일 경로
SAVE_DIR = os.path.join(PROJECT_ROOT, "FINE_TUNING_MODEL")
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8m_damage_stage2_3rd', 'weights', 'best.pt')

RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_finetuned_3rd_test_results_conf010.csv")

visualize_failures_batch(start_index=0, batch_size=100)

# 모델 학습 2차

In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# =========================================================
# [설정] 경로
# =========================================================
# 원본 (구글 드라이브)
SOURCE_DIR = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"
# 목적지 (Colab 로컬)
DEST_DIR = "/content/temp"

def copy_file(src_path, dst_path):
    shutil.copy2(src_path, dst_path)

def fast_copy_with_progress(src, dst):
    if not os.path.exists(src):
        print(f"❌ 원본 경로가 없습니다: {src}")
        return

    print("🔍 파일 목록을 스캔 중입니다... (잠시만 기다려주세요)")

    # 1. 모든 파일 리스트업
    files_to_copy = []
    for root, dirs, files in os.walk(src):
        # 목적지 폴더 구조 미리 생성
        rel_path = os.path.relpath(root, src)
        dest_path = os.path.join(dst, rel_path)
        os.makedirs(dest_path, exist_ok=True)

        for f in files:
            src_file = os.path.join(root, f)
            dst_file = os.path.join(dest_path, f)
            files_to_copy.append((src_file, dst_file))

    total_files = len(files_to_copy)
    print(f"📦 총 복사할 파일 수: {total_files}개")
    print(f"🚀 멀티쓰레딩 복사 시작 (8 workers)...")

    # 2. 병렬 복사 실행 (Thread Pool)
    with ThreadPoolExecutor(max_workers=8) as executor:
        # tqdm으로 진행바 연결
        list(tqdm(executor.map(lambda x: copy_file(x[0], x[1]), files_to_copy),
                  total=total_files,
                  unit='file'))

    print("\n✅ 데이터셋 복사 완료!")

# 실행
if os.path.exists(DEST_DIR):
    shutil.rmtree(DEST_DIR) # 기존꺼 삭제 후 재시작

fast_copy_with_progress(SOURCE_DIR, DEST_DIR)

In [ ]:

import yaml
# =========================================================
# 2. [설정] YAML 파일 경로 재지정
# =========================================================
# 로컬로 복사했으므로 YAML 파일 내의 path도 수정해줘야 함
yaml_path_local = os.path.join(DEST_DIR, 'data_damage_v2.yaml')

# YAML 내용 수정 (path: /content/temp_dataset 으로 변경)
with open(yaml_path_local, 'r') as f:
    data = yaml.safe_load(f)

data['path'] = DEST_DIR  # 경로를 로컬로 변경

with open(yaml_path_local, 'w') as f:
    yaml.dump(data, f)

print(f"📂 학습용 YAML 경로: {yaml_path_local}")

In [ ]:
import torch
import gc

# 메모리 청소
gc.collect()
torch.cuda.empty_cache()

In [ ]:

from ultralytics import YOLO

# =========================================================
# 3. [설정] 저장 경로 및 모델 학습
# =========================================================
PROJECT_DIR = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"
SAVE_DIR = os.path.join(PROJECT_DIR, "FINE_TUNING_MODEL")

def train_model():
    model = YOLO('yolov8x.pt')

    print("🔥 Stage 2 학습 시작...")
    model.train(
        data=yaml_path_local, # 로컬 YAML 사용
        epochs=100,
        imgsz=640,
        batch=16,

        # [튜닝 포인트 반영]
        patience=25,       # 10은 너무 짧습니다. 25로 늘리세요.
        optimizer='AdamW',
        lr0=1e-4,
        cos_lr=True,       # 학습률 스케줄러 추가 (권장)
        single_cls=True,   # 모든 라벨을 Damage 하나로 통합 (권장)

        close_mosaic=10,   # 오타 수정됨 (OK)

        project=SAVE_DIR,
        name='yolov8x_damage_stage2_4th',
        exist_ok=True,
        verbose=True
    )
    print("🎉 학습 완료!")
    return os.path.join(SAVE_DIR, 'yolov8x_damage_stage2_4th', 'weights', 'best.pt')

# 학습 실행
best_model_path = train_model()
print(f"🏆 Best Model Path: {best_model_path}")

## 성능 테스트(conf 0.25)

In [ ]:
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm

# [Cell 3] 성능 평가 (Detection & Classification)

best_model_path = '/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/FINE_TUNING_MODEL/yolov8x_damage_stage2_4th/weights/best.pt'
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"
# YAML 파일 경로
YAML_PATH = os.path.join(DATASET_ROOT, 'data_damage_v2.yaml')

def evaluate_performance(model_path):
    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_path}")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {model_path}")

    model = YOLO(model_path)

    # ---------------------------------------------------------
    # 1. 객체 탐지 성능 평가 (Detection Metrics: mAP)
    # ---------------------------------------------------------
    print("\n📊 [1] BBox 탐지 성능 (mAP) 계산 중...")
    # split='test'로 설정하여 Test Set에 대해 평가
    metrics = model.val(data=YAML_PATH, split='test', verbose=False)

    map50 = metrics.box.map50
    map5095 = metrics.box.map

    # ---------------------------------------------------------
    # 2. 이미지 분류 성능 & 속도 평가 (Classification & Speed)
    # ---------------------------------------------------------
    print("📊 [2] 이미지 분류 성능 및 추론 속도 측정 중...")

    TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
    TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    results_list = []
    speed_stats = []

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        txt_file = os.path.splitext(file)[0] + ".txt"
        lbl_path = os.path.join(TEST_LBL_DIR, txt_file)

        # A. 정답 확인 (라벨 파일 내용 유무)
        true_label = 0
        if os.path.exists(lbl_path) and os.path.getsize(lbl_path) > 0:
            true_label = 1 # Damaged

        # B. 모델 추론
        results = model(img_path, conf=0.25, verbose=False)

        # C. 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # D. 예측 라벨 (박스 유무)
        pred_label = 0
        if len(results[0].boxes) > 0:
            pred_label = 1 # Damaged

        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label)
        })

    df = pd.DataFrame(results_list)

    # =========================================================
    # [최종 분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # 1. mAP 결과 출력
    print(f"🎯 1. 객체 탐지 성능 (Detection Metrics):")
    print(f"   - mAP@50    : {map50:.4f}")
    print(f"   - mAP@50-95 : {map5095:.4f}")

    # 2. 정확도 출력
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"\n✅ 2. 분류 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # 3. 속도 출력

    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time if avg_time > 0 else 0

        print(f"\n⚡ 3. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # 4. 상세 리포트
    print(f"\n📝 4. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Normal', 'Damaged']))

    # 5. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Normal', 'Pred: Damaged'],
                yticklabels=['True: Normal', 'True: Damaged'])
    plt.title(f'Confusion Matrix\n(Damage Detection)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # 결과 저장
    save_path = os.path.join(RESULT_ROOT, "inference_finetuned_4th_test_results_conf025.csv")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 실행 결과 저장 완료: {save_path}")

# 평가 실행
evaluate_performance(best_model_path)


## 오탐 대상 시각화

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

# =========================================================
# [설정] 경로 (이전 단계와 동일하게 설정)
# =========================================================

DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"

# 모델 및 결과 파일 경로
SAVE_DIR = os.path.join(PROJECT_ROOT, "FINE_TUNING_MODEL")
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8x_damage_stage2_4th', 'weights', 'best.pt')

RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_finetuned_4th_test_results_conf025.csv")

# 테스트 이미지 및 라벨 경로
TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

def visualize_failures_batch(start_index=0, batch_size=3):
    """
    오탐 데이터를 start_index부터 batch_size만큼 시각화하는 함수
    """
    # 1. 결과 CSV 로드 및 오탐 필터링
    if not os.path.exists(RESULT_CSV_PATH):
        print(f"❌ 결과 파일이 없습니다: {RESULT_CSV_PATH}")
        return

    df = pd.read_csv(RESULT_CSV_PATH)
    failures = df[df['is_correct'] == False] # 오탐만 추출

    total_failures = len(failures)
    if total_failures == 0:
        print("🎉 오탐 데이터가 없습니다! 완벽합니다.")
        return

    # 인덱스 범위 체크
    if start_index >= total_failures:
        print(f"⚠️ 더 이상 확인할 오탐 데이터가 없습니다. (총 {total_failures}건)")
        return

    end_index = min(start_index + batch_size, total_failures)
    target_samples = failures.iloc[start_index : end_index]

    print(f"🔍 오탐 데이터 시각화: {start_index+1} ~ {end_index} / 총 {total_failures}건")

    # 2. 모델 로드 (박스 시각화용)
    model = YOLO(MODEL_PATH)

    # 3. 시각화
    plt.figure(figsize=(15, 6 * len(target_samples)))

    for i, (_, row) in enumerate(target_samples.iterrows()):
        filename = row['filename']
        true_label = row['true_label'] # 0: Normal, 1: Damaged
        img_path = os.path.join(TEST_IMG_DIR, filename)
        lbl_path = os.path.join(TEST_LBL_DIR, os.path.splitext(filename)[0] + ".txt")

        # 이미지 로드
        img = cv2.imread(img_path)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # A. 정답(Ground Truth) 박스 그리기 (초록색)
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls, cx, cy, bw, bh = map(float, line.strip().split())
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(img, "GT", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # B. 모델 예측(Prediction) 박스 그리기 (빨간색)
        # 시각화를 위해 다시 추론 수행
        results = model(img_path, conf=0.25, verbose=False)
        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
                cv2.putText(img, f"Pred {conf:.2f}", (x1, y2+25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # 타이틀 설정
        str_true = "Damaged" if true_label == 1 else "Normal"
        str_pred = "Normal" if true_label == 1 else "Damaged" # 오탐이므로 반대
        fail_type = "False Negative (미탐)" if true_label == 1 else "False Positive (오탐/과탐)"

        plt.subplot(len(target_samples), 1, i+1)
        plt.imshow(img)
        plt.title(f"[{start_index+i+1}] {filename}\nType: {fail_type} | GT: {str_true} (Green) vs Pred: {str_pred} (Red)",
                  fontsize=14, color='red', fontweight='bold')
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# 모델 및 결과 파일 경로
SAVE_DIR = os.path.join(PROJECT_ROOT, "FINE_TUNING_MODEL")
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8x_damage_stage2_4th', 'weights', 'best.pt')

RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_finetuned_4th_test_results_conf025.csv")

visualize_failures_batch(start_index=0, batch_size=100)

## 성능 테스트(conf 0.25->0.1)

In [ ]:
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm

# [Cell 3] 성능 평가 (Detection & Classification)

best_model_path = '/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/FINE_TUNING_MODEL/yolov8x_damage_stage2_4th/weights/best.pt'
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/DAMAGE_DETECTION_CLASS_ALL"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION"
# YAML 파일 경로
YAML_PATH = os.path.join(DATASET_ROOT, 'data_damage_v2.yaml')

def evaluate_performance(model_path):
    if not os.path.exists(model_path):
        print(f"❌ 모델 파일이 없습니다: {model_path}")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {model_path}")

    model = YOLO(model_path)

    # ---------------------------------------------------------
    # 1. 객체 탐지 성능 평가 (Detection Metrics: mAP)
    # ---------------------------------------------------------
    print("\n📊 [1] BBox 탐지 성능 (mAP) 계산 중...")
    # split='test'로 설정하여 Test Set에 대해 평가
    metrics = model.val(data=YAML_PATH, split='test', verbose=False)

    map50 = metrics.box.map50
    map5095 = metrics.box.map

    # ---------------------------------------------------------
    # 2. 이미지 분류 성능 & 속도 평가 (Classification & Speed)
    # ---------------------------------------------------------
    print("📊 [2] 이미지 분류 성능 및 추론 속도 측정 중...")

    TEST_IMG_DIR = os.path.join(DATASET_ROOT, 'images', 'test')
    TEST_LBL_DIR = os.path.join(DATASET_ROOT, 'labels', 'test')

    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    results_list = []
    speed_stats = []

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        txt_file = os.path.splitext(file)[0] + ".txt"
        lbl_path = os.path.join(TEST_LBL_DIR, txt_file)

        # A. 정답 확인 (라벨 파일 내용 유무)
        true_label = 0
        if os.path.exists(lbl_path) and os.path.getsize(lbl_path) > 0:
            true_label = 1 # Damaged

        # B. 모델 추론
        results = model(img_path, conf=0.1, verbose=False)

        # C. 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # D. 예측 라벨 (박스 유무)
        pred_label = 0
        if len(results[0].boxes) > 0:
            pred_label = 1 # Damaged

        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label)
        })

    df = pd.DataFrame(results_list)

    # =========================================================
    # [최종 분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # 1. mAP 결과 출력
    print(f"🎯 1. 객체 탐지 성능 (Detection Metrics):")
    print(f"   - mAP@50    : {map50:.4f}")
    print(f"   - mAP@50-95 : {map5095:.4f}")

    # 2. 정확도 출력
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"\n✅ 2. 분류 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # 3. 속도 출력

    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time if avg_time > 0 else 0

        print(f"\n⚡ 3. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # 4. 상세 리포트
    print(f"\n📝 4. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Normal', 'Damaged']))

    # 5. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Normal', 'Pred: Damaged'],
                yticklabels=['True: Normal', 'True: Damaged'])
    plt.title(f'Confusion Matrix\n(Damage Detection)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # 결과 저장
    save_path = os.path.join(RESULT_ROOT, "inference_finetuned_4th_test_results_conf010.csv")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 실행 결과 저장 완료: {save_path}")

# 평가 실행
evaluate_performance(best_model_path)


## 오탐 대상 시각화

In [ ]:
# 모델 및 결과 파일 경로
SAVE_DIR = os.path.join(PROJECT_ROOT, "FINE_TUNING_MODEL")
MODEL_PATH = os.path.join(SAVE_DIR, 'yolov8x_damage_stage2_4th', 'weights', 'best.pt')

RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK2_DAMAGE_DETECTION/RESULT"
RESULT_CSV_PATH = os.path.join(RESULT_ROOT, "inference_finetuned_4th_test_results_conf010.csv")

visualize_failures_batch(start_index=0, batch_size=100)